In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets

In [3]:
# load the CIFAR100 dataset
data_transforms = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize(
      mean = [0.485, 0.456, 0.406],   # Usimg mean and std dev of ImageNet
      std = [0.229, 0.224, 0.225])
])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=data_transforms)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

100%|██████████| 169001437/169001437 [00:03<00:00, 48958494.14it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data


In [4]:
# Create our own CNN

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # 3 convolutional layers
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)

        # 1 max pooling layer
        self.pool = nn.MaxPool2d(2, 2)

        # fully connected layers
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 100)  # 100 classes in CIFAR-100

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# call what we just defined to create a model
model = CNN()

In [5]:
# Create our loss and optimizer similar to part 1
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [7]:
# Training the CNN
for epoch in range(30):  # use 30 epochs
    print("Epoch", epoch+1, "/30")
    total_loss = 0.0    # track our loss as we train to make sure we are training effectively
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # tracking loss for testing purposes
    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(trainloader)}")

torch.save(model.state_dict(), 'trained_CNN.pth')

Epoch 1 /30
Epoch 1, Loss: 0.9157492529477
Epoch 2 /30
Epoch 2, Loss: 0.7951360586125528
Epoch 3 /30
Epoch 3, Loss: 0.6885850172289802
Epoch 4 /30
Epoch 4, Loss: 0.5783343438030509
Epoch 5 /30
Epoch 5, Loss: 0.48796228681455184
Epoch 6 /30
Epoch 6, Loss: 0.39647048868029316
Epoch 7 /30
Epoch 7, Loss: 0.33479650185236237
Epoch 8 /30
Epoch 8, Loss: 0.2718682424415408
Epoch 9 /30
Epoch 9, Loss: 0.23422965934723997
Epoch 10 /30
Epoch 10, Loss: 0.19208670676688253
Epoch 11 /30
Epoch 11, Loss: 0.167403253471798
Epoch 12 /30
Epoch 12, Loss: 0.1394134173312646
Epoch 13 /30
Epoch 13, Loss: 0.12454799231132278
Epoch 14 /30
Epoch 14, Loss: 0.10817771648054424
Epoch 15 /30
Epoch 15, Loss: 0.0961601715475378
Epoch 16 /30
Epoch 16, Loss: 0.06880717251635611
Epoch 17 /30
Epoch 17, Loss: 0.06478357745829941
Epoch 18 /30
Epoch 18, Loss: 0.04308659883568545
Epoch 19 /30
Epoch 19, Loss: 0.03821507364343566
Epoch 20 /30
Epoch 20, Loss: 0.0254538741595138
Epoch 21 /30
Epoch 21, Loss: 0.018709682711802633
E

In [8]:
# load model to predict labels for test set and print accuracy
testset = datasets.CIFAR100('.', train=False, transform = data_transforms, download=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

# Move the model to GPU (if available):
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Set model to evaluation / testing mode
model.eval()

# define vars for accuracy evaluation later
correct = 0
total = 0

with torch.no_grad():
  for i, data in enumerate(test_loader, 0):
    images, labels = data
    images, labels = images.to(device), labels.to(device)

    outputs = model(images)
    _, preds = torch.max(outputs, 1)
    correct += (preds == labels).sum().item()
    total += labels.size(0)

accuracy = (correct / total) * 100
print(accuracy)


100%|██████████| 169001437/169001437 [00:03<00:00, 48704279.80it/s]


Extracting ./cifar-100-python.tar.gz to .
39.92
